# <font color='red'>SEDES - Gerador de Dados</font>

# <font color='blue'>1.0 import</font>

In [1]:
import csv
import os
import pandas as pd
import random as rd
import datetime as dt
from dateutil.relativedelta import relativedelta

# <font color='blue'>2.0 parâmetros de ambiente</font>

In [2]:
amb_windows = False

HOME_PASTA = '' if amb_windows else os.path.expanduser("~")

In [3]:
# campo de negócio no BI
ORGAO_GESTOR = 'orgaoGestorDaPoliticaDeAssistenciaSocial'

# data inicial de simulação
DATA_INICIAL = (2018, 1, 1)

# tabelas gerais
n_orgao_gestor = (6, 11) # quantidade de orgãos gestores possíveis para a tabela de orgãos gestores

# formulários respondidos
form_resp = (80, 100) # % de formulários respondidos
orgao_gestor = (1, 3) # possibilidades de orgão gestor por município a cada data de referência
campos_por_form = (50, 150) # quantidade de campos por formulário sem contar os identificados para o BI

# <font color='blue'>3.0 extração</font>

## 3.1 extracao de dados externo

In [4]:
# abrindo os arquivos...
caminho_in = '../dados/apoio/'
caminho_out = '../dados/simulacao/'

arq_municipios = caminho_in + "tab_municipios.csv"
arq_instr_gestao = caminho_in + "tab_instr_gestao.csv"

df_municipios = pd.read_csv(arq_municipios, sep = ',', dtype = str)
df_instr_gestao = pd.read_csv(arq_instr_gestao, sep = ',', dtype = str)

print("municipios:              linhas = %s , colunas = %s " % df_municipios.shape)
print("instrumentos de gestão:  linhas = %s , colunas = %s " % df_instr_gestao.shape)

municipios:              linhas = 217 , colunas = 4 
instrumentos de gestão:  linhas = 5 , colunas = 1 


# <font color='blue'>4.0 transformação</font>

## 4.1 tabelas principais

### 4.1.1 tabela de datas dos protocolos

In [5]:
# cria a sequência de datas de referência passada
dt_inicial = dt.date(DATA_INICIAL[0], DATA_INICIAL[1], DATA_INICIAL[2])
lst_datas = []
dt_atual = dt_inicial

while dt_atual < dt.date.today():
    lst_datas.append(dt_atual)
    dt_atual = dt_atual + relativedelta(days = 1)
df_tb_datas = pd.DataFrame(lst_datas, columns=['dt_protocolo'])
df_tb_datas.shape

(1268, 1)

### 4.1.2 tabela de municípios

In [6]:
# campo identificado = municipio
df_tb_municipios = pd.DataFrame(df_municipios['city'])
df_tb_municipios.columns = ['municipio']
df_tb_municipios.shape

(217, 1)

### 4.1.3 tabela de "Orgãos Gestores"

In [7]:
# cria uma tabela de orgãos gestores possíveis
# quantidade de orgão gestores na tabela

lst_OG = ['Orgão Gestor ' + str(x + 1).zfill(2) for x in range(rd.randint(n_orgao_gestor[0], n_orgao_gestor[1]))]
lst_OG

['Orgão Gestor 01',
 'Orgão Gestor 02',
 'Orgão Gestor 03',
 'Orgão Gestor 04',
 'Orgão Gestor 05',
 'Orgão Gestor 06']

### 4.1.4 tabela de nomes de campos dos formulários

In [8]:
# gera nomes de colunas possíveis no set de dados
tot_cols = rd.randint(campos_por_form[0], campos_por_form[1])

lst = []
for i in range(tot_cols):
    lx = ['col_' + str(i), 'valor_col_' + str(i), 'STRING']
    lst.append(lx)
df_tb_colunas = pd.DataFrame(lst, columns=['atributo', 'valor', 'tipo'])
df_tb_colunas.head()

,atributo,valor,tipo
0,col_0,valor_col_0,STRING
1,col_1,valor_col_1,STRING
2,col_2,valor_col_2,STRING
3,col_3,valor_col_3,STRING
4,col_4,valor_col_4,STRING


# 4.2 grade de combinações genéricas

### 4.2.1 combinação Tabela de Datas x Colunas Diversas

In [11]:
# faz um cross merge entre datas e colunas diversas
df_aux = pd.merge(df_tb_datas, df_tb_colunas, how='cross')
df_aux


,dt_protocolo,atributo,valor,tipo
0,2018-01-01,col_0,valor_col_0,STRING
1,2018-01-01,col_1,valor_col_1,STRING
2,2018-01-01,col_2,valor_col_2,STRING
3,2018-01-01,col_3,valor_col_3,STRING
4,2018-01-01,col_4,valor_col_4,STRING
...,...,...,...,...
138207,2021-06-21,col_104,valor_col_104,STRING
138208,2021-06-21,col_105,valor_col_105,STRING
138209,2021-06-21,col_106,valor_col_106,STRING
138210,2021-06-21,col_107,valor_col_107,STRING


### 4.2.1 combinação Tabela de Datas x Tabela de Municípios

In [10]:
# faz um cross merge entre datas e municípios
df_aux = pd.merge(df_tb_datas, df_tb_municipios, how='cross')

# retira uma porcentagem da amostra
qtd_retira = rd.randint(form_resp[0], form_resp[1]) / 100

df_aux = df_aux.sample(frac=qtd_retira)

# gera um protocolo para cada combinação de dt_ref e município
df_aux['protocolo'] = df_aux.apply(lambda x: 'pct_' + str(x.name), axis=1)
print(df_aux.shape)
display(df_aux.head())

(239386, 3)


,dt_ref,municipio,protocolo
81933,2019-01-13,Montes Altos,pct_81933
125349,2019-08-01,Araioses,pct_125349
78015,2018-12-26,Santa Helena,pct_78015
255355,2021-03-22,Fortuna,pct_255355
210371,2020-08-27,Coroatá,pct_210371


### 4.2.2 acrescenta a dimensão colunas genéricas

In [11]:
# para cada protocolo gera linhas de colunas
df = pd.merge(df_aux, df_tb_colunas, how='cross')
print(df.shape)
display(df.head())

(13645002, 6)


,dt_ref,municipio,protocolo,atributo,valor,tipo
0,2019-01-13,Montes Altos,pct_81933,col_0,valor_col_0,STRING
1,2019-01-13,Montes Altos,pct_81933,col_1,valor_col_1,STRING
2,2019-01-13,Montes Altos,pct_81933,col_2,valor_col_2,STRING
3,2019-01-13,Montes Altos,pct_81933,col_3,valor_col_3,STRING
4,2019-01-13,Montes Altos,pct_81933,col_4,valor_col_4,STRING


# 4.3 grade de combinações de campos usados no BI

### 4.3.1 acrescenta campos "Orgão Gestos"

In [12]:
df_tmp = df_aux.copy()
df_tmp.shape

(239386, 3)

In [13]:
lst = []
for i in range(df_aux.shape[0]):
    lst.append([ORGAO_GESTOR, rd.sample(lst_OG, k=1)[0], 'STRING'])
df1 = pd.DataFrame(lst, columns=['atributo', 'valor', 'tipo'])
df1.shape

(239386, 3)

In [14]:
df2 = pd.concat([df_tmp.reset_index(drop=True), df1.reset_index(drop=True)], axis = 1)
df2.shape

(239386, 6)

In [15]:
df2

,dt_ref,municipio,protocolo,atributo,valor,tipo
0,2019-01-13,Montes Altos,pct_81933,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 11,STRING
1,2019-08-01,Araioses,pct_125349,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 06,STRING
2,2018-12-26,Santa Helena,pct_78015,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 05,STRING
3,2021-03-22,Fortuna,pct_255355,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 05,STRING
4,2020-08-27,Coroatá,pct_210371,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 03,STRING
...,...,...,...,...,...,...
239381,2019-01-20,Jenipapo dos Vieiras,pct_83507,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 03,STRING
239382,2020-10-01,Dom Pedro,pct_217919,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 03,STRING
239383,2020-04-17,Maracaçumé,pct_181721,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 09,STRING
239384,2018-04-11,Carutapera,pct_21814,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 07,STRING


In [16]:
df

,dt_ref,municipio,protocolo,atributo,valor,tipo
0,2019-01-13,Montes Altos,pct_81933,col_0,valor_col_0,STRING
1,2019-01-13,Montes Altos,pct_81933,col_1,valor_col_1,STRING
2,2019-01-13,Montes Altos,pct_81933,col_2,valor_col_2,STRING
3,2019-01-13,Montes Altos,pct_81933,col_3,valor_col_3,STRING
4,2019-01-13,Montes Altos,pct_81933,col_4,valor_col_4,STRING
...,...,...,...,...,...,...
13644997,2018-02-11,Jatobá,pct_9099,col_52,valor_col_52,STRING
13644998,2018-02-11,Jatobá,pct_9099,col_53,valor_col_53,STRING
13644999,2018-02-11,Jatobá,pct_9099,col_54,valor_col_54,STRING
13645000,2018-02-11,Jatobá,pct_9099,col_55,valor_col_55,STRING


In [17]:
df = pd.concat([df, df2])
df

,dt_ref,municipio,protocolo,atributo,valor,tipo
0,2019-01-13,Montes Altos,pct_81933,col_0,valor_col_0,STRING
1,2019-01-13,Montes Altos,pct_81933,col_1,valor_col_1,STRING
2,2019-01-13,Montes Altos,pct_81933,col_2,valor_col_2,STRING
3,2019-01-13,Montes Altos,pct_81933,col_3,valor_col_3,STRING
4,2019-01-13,Montes Altos,pct_81933,col_4,valor_col_4,STRING
...,...,...,...,...,...,...
239381,2019-01-20,Jenipapo dos Vieiras,pct_83507,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 03,STRING
239382,2020-10-01,Dom Pedro,pct_217919,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 03,STRING
239383,2020-04-17,Maracaçumé,pct_181721,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 09,STRING
239384,2018-04-11,Carutapera,pct_21814,orgaoGestorDaPoliticaDeAssistenciaSocial,Orgão Gestor 07,STRING


# <font color='blue'>5.0 carga</font>

In [23]:
# grava tabelas
df.to_csv(caminho_out + 'dataset_sedes.csv', index=False)

# TESTES

In [ ]:
location = pd.DataFrame({'area': ['new-york', 'columbo', 'mumbai']})
food = pd.DataFrame({'food': ['pizza', 'crabs', 'vada-paw']})

In [ ]:
location

In [ ]:
food

In [ ]:
pd.concat([location, food], join='outer', axis=1)

In [ ]:
det = pd.concat([location, food], join = 'outer', axis = 1)

In [ ]:
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'], index=[7, 8])
df1

In [ ]:
df4 = pd.DataFrame([['bird', 'polly'], ['monkey', 'george']], columns=['animal', 'name'])
df4

In [ ]:
pd.concat([df1, df4], axis=1)

In [ ]:
pd.concat([df1.reset_index(drop=True), df4.reset_index(drop=True)], axis=1)

In [ ]:
df4.reset_index(drop=True, inplace=True)
